In [3]:
import json
import requests

In [4]:
def read_airport_data_from_url(url):
    """Reads airport data from a JSON file at a given URL.

    Args:
        url: The URL of the JSON file.

    Returns:
        A dictionary containing the airport data, or None if an error occurs.
    """
    try:
        response = requests.get(url, timeout=10)  # Added timeout for robustness
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        airport_data = response.json()
        return airport_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format in the data from {url}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [5]:
url = "https://raw.githubusercontent.com/Jonty/airline-route-data/main/airline_routes.json"
airport_data = read_airport_data_from_url(url)

In [6]:
def get_airport_destinations(airport_data, airport_code):
    """Extracts destination airports for a given airport code.

    Args:
        airport_data: The loaded airport data (dictionary).
        airport_code: The IATA code of the airport.

    Returns:
        A set of destination airport IATA codes, or an empty set if the airport is not found or has no routes.
    """
    airport_info = airport_data.get(airport_code)
    if not airport_info:
        print(f"Airport {airport_code} not found in data.")
        return set()  # Return an empty set

    routes = airport_info.get('routes', [])  # Handle missing 'routes' key
    destinations = set()
    for route in routes:
        dest_iata = route.get('iata') # Handle missing 'iata' key in routes
        if dest_iata:
            destinations.add(dest_iata)
    return destinations


In [8]:
def find_common_destinations(airport_data, airport1, airport2):
    """Finds the common destination airports between two airports.

    Args:
        airport_data: The loaded airport data (dictionary).
        airport1: The IATA code of the first airport.
        airport2: The IATA code of the second airport.

    Returns:
        A set of common destination airport IATA codes.
    """
    destinations1 = get_airport_destinations(airport_data, airport1)
    destinations2 = get_airport_destinations(airport_data, airport2)

    common_destinations = destinations1.intersection(destinations2)
    return common_destinations

In [9]:
airport1_code = "SEA"  # Seattle-Tacoma International Airport
airport2_code = "BOS"  # Logan International Airport

common_destinations = find_common_destinations(airport_data, airport1_code, airport2_code)

In [10]:
common_destinations

{'AMS',
 'ATL',
 'AUS',
 'BNA',
 'BUF',
 'BWI',
 'BZN',
 'CDG',
 'CHS',
 'CLE',
 'CLT',
 'CMH',
 'CUN',
 'CVG',
 'DAL',
 'DCA',
 'DEN',
 'DFW',
 'DOH',
 'DTW',
 'DUB',
 'DXB',
 'EWR',
 'FLL',
 'FRA',
 'HDN',
 'HNL',
 'IAD',
 'IAH',
 'ICN',
 'IND',
 'IST',
 'JFK',
 'KEF',
 'LAS',
 'LAX',
 'LHR',
 'LIR',
 'MCI',
 'MCO',
 'MDT',
 'MDW',
 'MEX',
 'MIA',
 'MKE',
 'MSP',
 'MSY',
 'MUC',
 'NAS',
 'NRT',
 'ORD',
 'PDX',
 'PEK',
 'PHL',
 'PHX',
 'PIT',
 'RDU',
 'RSW',
 'SAN',
 'SAT',
 'SFO',
 'SJC',
 'SLC',
 'SMF',
 'STL',
 'TPA',
 'YUL',
 'YVR',
 'YYC',
 'YYZ',
 'ZRH'}

In [ ]:

if __name__ == "__main":
    # Example usage:
    url = "https://raw.githubusercontent.com/Jonty/airline-route-data/main/airline_routes.json"
    airport_data = read_airport_data_from_url(url)

    if airport_data:
        airport1_code = "SEA"  # Seattle-Tacoma International Airport
        airport2_code = "BOS"  # Logan International Airport

        common_destinations = find_common_destinations(airport_data, airport1_code, airport2_code)

        if common_destinations:
            print(f"Common destinations between {airport1_code} and {airport2_code}:")
            for dest in common_destinations:
                print(dest)
        else:
            print(f"No common destinations found between {airport1_code} and {airport2_code}.")
else:
    print("Failed to load airport data.")

In [15]:
import pandas as pd

def extract_airport_data(airport_dict):
    """Extracts IATA, name, and display_name from an airport dictionary and returns a Pandas DataFrame.

    Args:
        airport_dict: A dictionary where keys are airport codes and values are airport information dictionaries.

    Returns:
        A Pandas DataFrame containing the extracted data, or an empty DataFrame if the input is invalid.
    """

    data = []
    for airport_code, airport_info in airport_dict.items():
        iata = airport_info.get('iata')  # Use .get() to handle missing keys safely
        name = airport_info.get('name')
        display_name = airport_info.get('display_name')

        if iata and name and display_name: # Check if the required values are present
            data.append({'iata': iata, 'name': name, 'display_name': display_name})
        else:
            print(f"Warning: Incomplete data for airport code: {airport_code}. Skipping.")

    df = pd.DataFrame(data)
    return df

In [16]:
airport_df = extract_airport_data(airport_data)

In [ ]:
data = {'iata': list(common_destinations)}  # Create a dictionary for the DataFrame
df = pd.DataFrame(data)
common_destinations_df = df.merge(airport_df, how='left', on='iata')

In [23]:
common_destinations_df.head()

,iata,name,display_name
0,HNL,Honolulu International Airport,"Honolulu (HNL), USA"
1,KEF,Keflavík International Airport,"Reykjavik (KEF), Iceland"
2,EWR,Newark Liberty International,"New York (EWR), USA"
3,ICN,Incheon International Airport,"Seoul (ICN), Republic of Korea"
4,CLT,Charlotte Douglas International,"Charlotte (CLT), USA"


In [24]:
def get_flight_times_df(airport_data, airport1_code, airport2_code, common_destinations):
    """Gets flight times for common destinations and returns a Pandas DataFrame.

    Args:
        airport_data: The loaded airport data (dictionary).
        airport1_code: IATA code of the first airport.
        airport2_code: IATA code of the second airport.
        common_destinations: A set of common destination IATA codes.

    Returns:
        A Pandas DataFrame with columns 'iata', airport1_code, and airport2_code, or an empty DataFrame if no data is found.
    """

    data = []
    for dest in common_destinations:
        time1 = None
        time2 = None

        origin1_info = airport_data.get(airport1_code)
        if origin1_info:
          routes1 = origin1_info.get('routes', [])
          for route in routes1:
              if route.get('iata') == dest:
                  time1 = route.get('min')
                  break  # Stop searching once time is found

        origin2_info = airport_data.get(airport2_code)
        if origin2_info:
          routes2 = origin2_info.get('routes', [])
          for route in routes2:
              if route.get('iata') == dest:
                  time2 = route.get('min')
                  break  # Stop searching once time is found

        data.append({'iata': dest, airport1_code: time1, airport2_code: time2})

    df = pd.DataFrame(data)
    return df

In [25]:
df_flights = get_flight_times_df(airport_data, airport1_code, airport2_code, common_destinations)

In [29]:
common_destinations_df = df_flights.merge(airport_df, how='left', on='iata')
common_destinations_df['travel_diff_min'] = common_destinations_df.iloc[:, 1] - common_destinations_df.iloc[:, 2]

In [30]:
common_destinations_df.head()

,iata,SEA,BOS,name,display_name,travel_diff_min
0,HNL,377,675,Honolulu International Airport,"Honolulu (HNL), USA",-298
1,KEF,445,315,Keflavík International Airport,"Reykjavik (KEF), Iceland",130
2,EWR,322,88,Newark Liberty International,"New York (EWR), USA",234
3,ICN,700,920,Incheon International Airport,"Seoul (ICN), Republic of Korea",-220
4,CLT,305,152,Charlotte Douglas International,"Charlotte (CLT), USA",153


In [32]:
min_travel = common_destinations_df['travel_diff_min'].abs().min()
common_destinations_df[common_destinations_df['travel_diff_min']==min_travel]

,iata,SEA,BOS,name,display_name,travel_diff_min
44,MSP,202,197,Minneapolis St Paul International,"Minneapolis (MSP), USA",5


In [34]:
common_destinations_df.to_csv('common_destinations.csv', index=False)